# Lesson 4: Chaining Prompts for Agentic Reasoning

## Automated Claim Triage: From First-Notice to the Right Queue

In this hands-on exercise, you will build a prompt chain that extracts key fields from free-form auto-claim reports, assesses damage severity, and routes each claim to one of several queues — `glass`, `fast_track`, `material_damage`, or `total_loss` — with code-based gate checks at every step.

## Outline:

- Setup
- Sample FNOL (First Notice of Loss) Texts
- Stage I: Information Extraction
- Stage II: Severity Assessment
- Stage III: Queue Routing
- Review Outputs

## Setup

Import necessary libraries and define helper functions, including a mock LLM client, code execution environment, and test runner.

In [1]:
# Import necessary libraries
# No changes needed in this cell
from openai import OpenAI  # For accessing the OpenAI API
from enum import Enum
import json
from pydantic import BaseModel, Field  # For structured data validation
from typing import List, Literal, Optional

In [3]:
# Set up LLM credentials

client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    api_key="voc-18578903711607364724451696893a3acedc2.01924345",  # <--- TODO: Fill in your Vocareum API key here
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)

# If using OpenAI's API endpoint
# client = OpenAI()

In [4]:
# Define helper functions
# No changes needed in this cell


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO


def get_completion(messages=None, system_prompt=None, user_prompt=None, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """

    messages = list(messages)
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message.content

## Sample FNOL (First Notice of Loss) Texts
Let's define a few sample First Notice of Loss (FNOL) texts to process through our chain.

In [5]:
# Define sample FNOL texts
# TODO: [Optional] Add more sample FNOL texts to test various scenarios

sample_fnols = [
    """
    Claim ID: C001
    Customer: John Smith
    Vehicle: 2018 Toyota Camry
    Incident: While driving on the highway, a rock hit my windshield and caused a small chip
    about the size of a quarter. No other damage was observed.
    """,
    """
    Claim ID: C002
    Customer: Sarah Johnson
    Vehicle: 2020 Honda Civic
    Incident: I was parked at the grocery store and returned to find someone had hit my car and
    dented the rear bumper and taillight. The taillight is broken and the bumper has a large dent.
    """,
    """
    Claim ID: C003
    Customer: Michael Rodriguez
    Vehicle: 2022 Ford F-150
    Incident: I was involved in a serious collision at an intersection. The front of my truck is
    severely damaged, including the hood, bumper, radiator, and engine compartment. The airbags
    deployed and the vehicle is not drivable.
    """,
    """
    Claim ID: C004
    Customer: Emma Williams
    Vehicle: 2019 Subaru Outback
    Incident: My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and
    trunk. The side mirrors were also damaged and one window has a small crack.
    """,
    """
    Claim ID: C005
    Customer: David Brown
    Vehicle: 2021 Tesla Model 3
    Incident: Someone keyed my car in the parking lot. There are deep scratches along both doors
    on the driver's side.
    """,
]

## Stage I: Information Extraction
In this stage, we'll create a prompt that extracts structured information from free-form FNOL text.

In [7]:
# Define a system prompt for information extraction according to the provided ClaimInformation class
# TODO: Complete the prompt by replacing the parts marked with **********


class ClaimInformation(BaseModel):
    claim_id: str = Field(..., min_length=2, max_length=10)
    name: str = Field(..., min_length=2, max_length=100)
    vehicle: str = Field(..., min_length=2, max_length=100)
    loss_desc: str = Field(..., min_length=10, max_length=500)
    damage_area: List[
        Literal[
            "windshield",
            "front",
            "rear",
            "side",
            "roof",
            "hood",
            "door",
            "bumper",
            "fender",
            "quarter panel",
            "trunk",
            "glass",
        ]
    ] = Field(..., min_length=1)


info_extraction_system_prompt = """
You are an auto insurance claim processing assistant. Your task is to extract key information from First Notice of Loss (FNOL) reports.

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID
- name (str): The customer's full name
- vehicle (str): The vehicle make and model
- loss_desc (str): A brief description of the loss incident
- damage_area (List[str]): A list of damaged areas of the vehicle. Possible values include:
  - windshield
  - front
  - rear
  - side
  - roof
  - hood
  - door
  - bumper
  - fender
  - quarter panel
  - trunk
  - glass

For damage_area, only use items from the list above.

Only respond with the JSON object, nothing else.
"""

In [8]:
# Define a gate check function and claim extraction function
# TODO: Complete the prompt by replacing the parts marked with **********


def gate1_validate_claim_info(claim_info_json: str) -> ClaimInformation:
    """
    Gate 1: Validates claim information extracted from FNOL text.
    Returns validated ClaimInformation object or raises validation error.
    """
    try:
        # Parse the JSON string
        claim_info_dict = json.loads(claim_info_json)
        # Validate with Pydantic model
        validated_info = ClaimInformation(**claim_info_dict)
        return validated_info
    except Exception as e:
        raise ValueError(f"Gate 1 validation failed: {str(e)}")


def extract_claim_info(fnol_text):
    """
    Stage 1: Extract structured information from FNOL text
    """
    messages = [
        {"role": "system", "content": info_extraction_system_prompt},
        {"role": "user", "content": fnol_text},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the extracted information
    try:
        validated_info = gate1_validate_claim_info(response)
        return validated_info
    except ValueError as e:
        print(f"Gate 1 failed: {e}")
        return None

In [9]:
# Run the claim extraction function on the sample FNOLs
# No updates needed in this cell

extracted_claim_info_items = [
    extract_claim_info(fnol_text) for fnol_text in sample_fnols
]
extracted_claim_info_items

[ClaimInformation(claim_id='C001', name='John Smith', vehicle='2018 Toyota Camry', loss_desc='While driving on the highway, a rock hit my windshield and caused a small chip about the size of a quarter. No other damage was observed.', damage_area=['windshield']),
 ClaimInformation(claim_id='C002', name='Sarah Johnson', vehicle='2020 Honda Civic', loss_desc='Someone hit the parked car, damaging the rear bumper and taillight', damage_area=['bumper', 'rear']),
 ClaimInformation(claim_id='C003', name='Michael Rodriguez', vehicle='2022 Ford F-150', loss_desc='Serious collision at intersection with severe damage to the front, including hood, bumper, radiator, and engine compartment. Airbags deployed, vehicle not drivable.', damage_area=['front', 'hood', 'bumper']),
 ClaimInformation(claim_id='C004', name='Emma Williams', vehicle='2019 Subaru Outback', loss_desc='Damaged in a hailstorm with dents on hood, roof, trunk, damaged side mirrors, and a cracked window', damage_area=['hood', 'roof', 't

## Stage II: Severity Assessment
In this stage, we'll assess the severity of the damage based on the extracted information.

Note, our carrier applies the following heuristics:
- Minor damage: Small dents, scratches, glass chips (cost range: $100-$1,000)
- Moderate damage: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
- Major damage: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)

In this example we will let the LLM estimate the cost, though in production we would want a more accurate estimate, e.g. querying a database of repair costs.


In [10]:
# Define a system prompt for severity assessment according to the provided SeverityAssessment class
# TODO: Complete the prompt by replacing the parts marked with **********


class SeverityAssessment(BaseModel):
    severity: Literal["Minor", "Moderate", "Major"]
    est_cost: float = Field(..., gt=0)


severity_assessment_system_prompt = """
You are an auto insurance damage assessor. Your task is to evaluate the severity of vehicle damage and estimate repair costs.

Apply these carrier heuristics:
- Minor damage: Cosmetic issues like scratches, small dents, or minor glass chips. Estimated cost: <$1,000.
- Moderate damage: Functional damage that affects vehicle operation but is repairable, such as broken lights
- Major damage: Severe damage affecting vehicle structure or safety, such as frame damage or engine issues. Estimated cost: >$5,000.

Based on the provided claim information, assess the severity of the damage and estimate the repair cost.
1. Severity levels:
   - Minor
   - Moderate
   - Major
2. Estimated repair cost in USD.

Format your response as a valid JSON object with the following keys:
- severity (str): One of "Minor", "Moderate", or "Major"
- est_cost (float): Estimated repair cost in USD (e.g., 2500.00)

Only respond with the JSON object, nothing else.
"""

In [11]:
# Define a gate check function and assess_severity function
# TODO: Complete the prompt by replacing the parts marked with **********


def gate2_cost_range_ok(severity_json: str) -> SeverityAssessment:
    """
    Gate 2: Validates that the estimated cost is within reasonable range for the severity.
    Returns validated SeverityAssessment object or raises validation error.
    """
    try:
        # Parse the JSON string
        severity_dict = json.loads(severity_json)
        # Validate with Pydantic model
        validated_severity = SeverityAssessment(**severity_dict)

        # Check cost range based on severity
        if (
            validated_severity.severity == "Minor"
            and (
                validated_severity.est_cost > 1000
            )
        ):
            raise ValueError(
                f"Minor damage should cost between $100-$1000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Moderate"
            and (
                validated_severity.est_cost < 1000 or validated_severity.est_cost > 5000
            )
        ):
            raise ValueError(
                f"Moderate damage should cost between $1000-$5000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Major"
            and (
                validated_severity.est_cost < 5000 or validated_severity.est_cost > 50000
            )
        ):
            raise ValueError(
                f"Major damage should cost between $5000-$50000, got ${validated_severity.est_cost}"
            )


        return validated_severity
    except Exception as e:
        raise ValueError(f"Gate 2 validation failed: {str(e)}")


def assess_severity(claim_info: ClaimInformation) -> Optional[SeverityAssessment]:
    """
    Stage 2: Assess severity based on damage description
    """

    # Convert Pydantic model to JSON string
    claim_info_json = claim_info.model_dump_json()

    messages = [
        {"role": "system", "content": severity_assessment_system_prompt},
        {"role": "user", "content": claim_info_json},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the severity assessment
    try:
        validated_severity = gate2_cost_range_ok(response)
        return validated_severity
    except ValueError as e:
        print(f"Gate 2 failed: {e}. Response: {response}")
        return None


In [12]:
# Run the claim extraction function on the sample data
# No updates needed in this cell

severity_assessment_items = [
    assess_severity(item) for item in extracted_claim_info_items
]

severity_assessment_items

[SeverityAssessment(severity='Minor', est_cost=200.0),
 SeverityAssessment(severity='Moderate', est_cost=1500.0),
 SeverityAssessment(severity='Major', est_cost=15000.0),
 SeverityAssessment(severity='Moderate', est_cost=3000.0),
 SeverityAssessment(severity='Minor', est_cost=500.0)]

## 6. Stage III: Queue Routing
In this stage, we'll route the claim to the appropriate queue based on severity and damage area.

Use these routing rules:
- 'glass' queue: For Minor damage involving ONLY glass (windshield, windows)
- 'fast_track' queue: For other Minor damage
- 'material_damage' queue: For all Moderate damage
- 'total_loss' queue: For all Major damage

These are the priority levels:
- Priority 1 (highest): Safety issues, customer stranded
- Priority 2: Significant but contained damage, vehicle drivable
- Priority 3: Standard claims
- Priority 4: Minor issues, no mobility impact
- Priority 5 (lowest): Cosmetic only, no functional impact

In [16]:
# Define a system prompt for claim routing according to the provided ClaimRouting class
# TODO: Complete the prompt by replacing the parts marked with **********


class ClaimRouting(BaseModel):
    claim_id: str
    queue: Literal["glass", "fast_track", "material_damage", "total_loss"]
    priority: int = Field(..., ge=1, le=5)


queue_routing_system_prompt = """
You are an auto insurance claim routing specialist. Your task is to determine the appropriate processing queue for each claim.

Use the following rules to assign the queue:
- glass: If the damage is limited to glass components (e.g., windshield, windows)
- fast_track: If the damage is minor (estimated cost <$1,000) and does
- material_damage: If the damage is moderate or major but the vehicle is repairable
- total_loss: for all major damege

Also assign a priority level:
- Priority 1: Highest priority, safety issues, customer stranded
- Priority 2: Significant but contained damage, vehicle drivable
- Priority 3: Standard claims
- Priority 4: Minor issues, no mobility impact
- Priority 5: Lowest priority, minor cosmetic issues

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID
- queue (str): One of "glass", "fast_track", "material_damage", or "total_loss"
- priority (int): An integer from 1 (highest) to 5 (lowest)

Only respond with the JSON object, nothing else.
"""

In [17]:
# Define a gate check function and assess_severity function
# No updates needed in this cell


def gate3_validate_routing(routing_json: str) -> ClaimRouting:
    """
    Gate 3: Validates that the claim is routed to a valid queue.
    Returns validated ClaimRouting object or raises validation error.
    """
    try:
        # Parse the JSON string
        routing_dict = json.loads(routing_json)
        # Validate with Pydantic model
        validated_routing = ClaimRouting(**routing_dict)
        return validated_routing
    except Exception as e:
        raise ValueError(f"Gate 3 validation failed: {str(e)}")


def route_claim(
    claim_info: ClaimInformation, severity_assessment: Optional[SeverityAssessment]
) -> Optional[ClaimRouting]:
    """
    Stage 3: Route claim to appropriate queue
    """
    if severity_assessment is None:
        return None

    # Create input for the routing model
    routing_input = {
        "claim_info": claim_info.model_dump(),
        "severity_assessment": severity_assessment.model_dump(),
    }

    messages = [
        {"role": "system", "content": queue_routing_system_prompt},
        {"role": "user", "content": json.dumps(routing_input)},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the routing decision
    try:
        validated_routing = gate3_validate_routing(response)
        return validated_routing
    except ValueError as e:
        print(f"Gate 3 failed: {e}. Response: {response}")
        return None

In [18]:
# Run the routing function on the sample data
# No updates needed in this cell

routed_claim_items = [
    route_claim(claim, severity_assessment)
    for claim, severity_assessment in zip(
        extracted_claim_info_items, severity_assessment_items
    )
]

routed_claim_items

[ClaimRouting(claim_id='C001', queue='glass', priority=4),
 ClaimRouting(claim_id='C002', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C003', queue='total_loss', priority=1),
 ClaimRouting(claim_id='C004', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C005', queue='fast_track', priority=4)]

## 7. Review Outputs

Let's put our data into a pandas dataframe for easier analysis.

In [19]:
# No updates needed in this cell

import pandas as pd

records = []
for claim, severity_assessment, routed_claim in zip(
    extracted_claim_info_items, severity_assessment_items, routed_claim_items
):
    record = {}
    record.update(claim)
    record.update(severity_assessment)
    record.update(routed_claim)
    records.append(record)


# Show the entire dataframe since it is not too large
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(records)

df

ModuleNotFoundError: No module named 'pandas'

## 8. Reflection & Transfer
Reflect on the effectiveness of chaining prompts for this task:

1. Prompt Chain Architecture:
* How does breaking down the task into stages affect the performance?
* What are the benefits of having gate checks between stages?
* How could the chain design be improved?

2. Error Handling:
* What types of errors might occur at each stage?
* How could we make the chain more robust against errors?
* What would a good fallback strategy look like?

3. Scalability:
* How well would this approach scale to handle more complex claims?
* What challenges might arise when processing thousands of claims?
* How could the prompt chain be optimized for efficiency?

4. Transfer to Other Domains:
* How could this prompt chaining approach be applied to other domains?
* What general principles of prompt chaining can we extract from this exercise?

## Summary

🎉 Congratulations! 🎉 You've built an impressive prompt chain system for insurance claims!
You transformed messy FNOL text into structured data, assessed damage severity, and routed claims to the right queues, all with robust gate checks! 🚀✨

Remember:

- 🔗 Chained prompts break complex tasks into manageable steps
- 🛡️ Gate checks prevent error cascades
- 🧠 Having specialized prompts helps keep code focused and maintainable

You've mastered a powerful pattern for countless business processes! 🏆
Amazing work on your agentic reasoning system! 💯